# OLS推定量の性質

## 重回帰モデル

$N$個のサンプルがあり、$i$番目のサンプルについての回帰式を次のように表記する

$$
Y_i = X_i^T \beta + u_i, \hspace{1em} i=1,\dots, N
$$

ここで$X_i \in \mathbb{R}^D$は説明変数の行列$X\in \mathbb{R}^{N\times D}$を1行取り出したもので、$Y_i, u_i \in \mathbb{R}$も1つのサンプルの被説明変数と誤差項である。


行列表記にすると

$$
Y = X \beta + u
$$

と表すことができる。


### OLS推定量

目的関数は残差の二乗和であるため、

$$
\begin{align}
L(\beta) &= u^T u\\
&= (Y - X \beta)^T (Y - X \beta)\\
&= Y^T Y - Y^T X \beta - (X\beta)^T Y + (X \beta)^T X \beta\\
&= Y^T Y - 2 \beta^T X^T Y + \beta^T X^T X \beta\\
\end{align}
$$

:::{margin} 式展開メモ
- $(X \beta)^T = \beta^T X^T$ のため
- $Y^T X \beta$と$(X\beta)^T Y$は同値のスカラーになるので$2 \beta^T X^T Y$と1つの項にまとめた
:::


である。これを微分してゼロとおくと

$$
\frac{\partial L(\beta)}
{\partial \beta}
=  - 2 X^T Y + 2 X^T X \beta
= 0
$$

となり、$\beta$について解くと

$$
\beta
= (X^T X)^{-1} X^T Y
$$

:::{margin} 式展開メモ
ベクトル$x$、行列$A$に対し

$$
\frac{\partial A x}{\partial x^T} = A
$$

$$
\frac{\partial x^T B x}{\partial x} = (B + B^T) x
$$

もしBが対称行列なら$(B + B^T) x = 2Bx$
:::



In [35]:
import numpy as np

x0 = np.array([1, 1, 1])
x1 = np.array([1, 2, 3])
x2 = np.array([2, 8, 9])
X = np.array([x0, x1, x2]).T
beta = np.array([3, 5, 7]) # 真のbeta
y = X @ beta

# OLS推定量
beta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)
beta_hat.round(1)

array([3., 5., 7.])

## OLS推定量の別表記

一致性や不偏性の議論のための準備として、OLS推定量を変形する。

重回帰モデル$Y = X\beta + u$をOLS推定量$\hat{\beta} = (X^T X)^{-1} X^T Y$に代入して変形すると

$$
\begin{align}
\hat{\beta}
&= (X^T X)^{-1} X^T Y\\
&= (X^T X)^{-1} X^T (X\beta + u)\\
&= (X^T X)^{-1} X^T X\beta + (X^T X)^{-1} X^T u\\
&= \beta + (X^T X)^{-1} X^T u\\
&= \beta + \left(\frac{1}{N} X^T X\right)^{-1} \frac{1}{N} X^T u
\end{align}
$$

となる。

単回帰でいうと

$$
\hat{\beta} = \beta + \frac{Cov(X, u)}{Var(X)}
$$

である。$X$に内生性がある、すなわち$Cov(X, u) \neq 0$であると$\hat{\beta} \neq \beta$となる


### OLS推定量のバリアンス

$\hat{\beta} = \beta + (X^T X)^{-1} X^T u$ は $u\sim N(0, \sigma^2 I)$の仮定より、$\hat{\beta} \sim N(\beta, \sigma^2 (X^\top X)^{-1})$となる。

よって $\operatorname{Var}[\hat{\beta}] = \sigma^2 (X^\top X)^{-1}$ となる

::::{dropdown} 証明

以下の定理を使う

:::{card} 定理

$u$を確率変数ベクトルとし、$\mu\in\mathbb{R}^n$、$b\in\mathbb{R}^p$、$C\in\mathbb{R}^{p\times n}$、$\operatorname{rank} C = p$とする。

$u \sim N(\mu, \Sigma)$ のとき、$Cu + b \sim N(C\mu + b, C\Sigma C^\top)$

:::

$C := (X^T X)^{-1} X^T$とおけば

$$
\begin{aligned}
C^T = [(X^T X)^{-1} X^T]^T
&= X [(X^T X)^{-1}]^T \quad (\because (AB)^T = B^T A^T) \\
&= X [(X^T X)^T]^{-1} \quad (\because (A^{-1})^T = (A^T)^{-1} ) \\
&= X (X^T X)^{-1} \quad (\because (X^T X)^T=X^T X ) \\
\end{aligned}
$$

であるため、$u\sim N(0, \sigma^2 I)$ の仮定が満たされるとき、

$$
\begin{aligned}
&\beta + C u \sim N(\beta, \sigma^2 CC^T)\\
&= \beta + (X^T X)^{-1} X^T u \sim 
N(\beta, \sigma^2 (X^T X)^{-1} X^T X (X^T X)^{-1} )\\
&= \beta + (X^T X)^{-1} X^T u \sim 
N(\beta, \sigma^2 (X^T X)^{-1})
\end{aligned}
$$

:::

:::{admonition} OLSの仮定
1. i.i.d.：$(Y, X)$は独立同一分布に従う
2. 外生性：$E[u|X] = 0$
3. 異常値がない：$X, u$は4次までのモーメントを持つ
4. 多重共線性がない：任意の$\sum_{j=0}^k a_j^2 = 1$となる$a_0,\dots,a_k$について$E[(a_0 + a_1 X_1 + \cdots + a_k X_k)^2]>0$が成り立つ
:::

## 不偏性

### 外生性

単回帰モデル$Y = \alpha + \beta X + u$を例にとる。

:::{admonition} 外生性

説明変数$X$と誤差項$u$が

$$
E(u | X) = 0
$$

を満たすとき、$X$は外生変数であるという。
:::

また、外生性の条件は別の表現もできる

:::{admonition} $X$と$u$の直交
説明変数$X$が外生変数ならば、

$$
E(u) = 0, E(X, u) = 0
$$
:::

:::{dropdown} 証明
$$
E(X u) = E_{X}[ E(X u | X) ] = E_{X}[ X \underbrace{ E(u|X) }_{ =0 } ] = E_{X}(X\cdot 0) = 0
\\
E(u) = E_{X}[ \underbrace{ E(u|X) }_{ =0 } ] = 0
$$
:::

さらに、共分散との関係も導出できる

:::{admonition} $X$と$u$の無相関
説明変数$X$が外生変数ならば、

$$
Cov(X, u) = 0
$$
:::

:::{dropdown} 証明

$$
Cov(X, u) = \underbrace{ E(X u) }_{ =0 }
- E(X) \underbrace{ E(u) }_{ =0 }
= 0
$$
:::

### OLS推定量の不偏性

単回帰モデル$Y = \alpha + \beta X + u$のOLS推定量$\beta$の確率極限は

$$
\DeclareMathOperator{\cov}{\text{Cov}}
\DeclareMathOperator{\var}{\text{Var}}
\DeclareMathOperator{\plim}{\text{plim}}
\plim \beta = \beta + \frac{\cov(X, u)}{\var(X)}
$$

となる。外生性が満たされるとき$\cov(X, u) = 0$であるため、$\plim \beta = \beta$となり、OLS推定量は母回帰係数の不偏推定量となる。

::::{dropdown} 証明
$$
\DeclareMathOperator{\cov}{\text{Cov}}
\DeclareMathOperator{\var}{\text{Var}}
\DeclareMathOperator{\plim}{\text{plim}}
\begin{align}
\plim \beta
&= \frac{\cov(X, Y)}{\var(X)}\\
&= \frac{\cov(X, \alpha + \beta X + u)}{\var(X)}\\
&= \frac{\cov(X, \alpha) + \cov(X, \beta X) + \cov(X, u) }{\var(X)}\\
&= \frac{\beta \var(X) + \cov(X, u) }{\var(X)}\\
&= \beta + \frac{\cov(X, u)}{\var(X)}
\end{align}
$$


:::{admonition}（参考）和の共分散
標本値、確率変数の和は、加える前の個々の共分散の和になる。すなわち、共分散においては分配法則が成り立つ。

$$
\cov(X + Z, Y) = \cov(X, Y) + \cov(Z, Y)
$$

参考：[確率統計 – 分散と共分散 – TauStation](http://taustation.com/statistics-variance-and-covariance/)
:::

:::{admonition} （参考）共分散と定数

$\alpha, \beta$は定数と考えると、

$$
\cov(X, \alpha)
= E(X \alpha) - E(X) E(\alpha)\\
= \alpha E(X) - \alpha E(X)\\
= 0
$$

であり

$$
\cov(X, \beta X)
= E(X \beta X) - E(X) E(\beta X)\\
= \beta E(X^2) - \beta E(X)^2\\
= \beta \var(X)
$$
:::
::::

## 一致性

異常値がない（$X, u$は4次までのモーメントを持つ）という仮定と大数の法則により以下が成立する

$$
\begin{align}
\frac{1}{N} X^T X
    &= \frac{1}{N} \sum^N_{i=1} X_i X_i^T
    \overset{p}{\longrightarrow}
    E(X_i X_i^T)
\\
\frac{1}{N} X^T u
    &= \frac{1}{N} \sum^N_{i=1} X_i u_i
    \overset{p}{\longrightarrow}
    E(X_i u_i)
    = 0
\end{align}
$$

多重共線性がないという仮定により$(E(X_i X_i^T))^{-1}$が存在する

$$
\left(
    \frac{1}{N} X^T X
\right)^{-1}
\frac{1}{N} X^T u
\overset{p}{\longrightarrow}
0
$$

よって

$$
\hat{\beta}
\overset{p}{\longrightarrow}
\beta
$$


## 漸近正規性

OLS推定量

$$
\hat{\beta}
= \beta + \left(\frac{1}{N} X^T X\right)^{-1} \frac{1}{N} X^T u
$$

を整理して以下の形にする

$$
\sqrt{N} (\hat{\beta} - \beta)
=
\left(
    \frac{1}{N} X^T X
\right)^{-1}
\frac{1}{\sqrt{N}} X^T u
$$

$\frac{1}{\sqrt{N}} X^T u$は$\frac{1}{\sqrt{N}} \sum^N_{i=1} X_i u_i$と書くことができる。OLSの仮定より

$$
\begin{align}
E(X_i u_i) &= 0\\
Var(X_i u_i) &= E(u^2 X_i X^T_i)
\end{align}
$$

なので、中心極限定理により

$$
\frac{1}{\sqrt{N}} X^T u
= \frac{1}{\sqrt{N}} \sum^N_{i=1} X_i u_i
 \overset{d}{\longrightarrow} N\left( 0, E(u_i^2 X_i X_i^T) \right)
$$

となる。

一致性のときに導出した

$$
\frac{1}{N} X^T X
\overset{p}{\longrightarrow} E(X_i X_i^T)
$$

を使うと、スルツキーの定理を用いて

$$
\begin{align}
\sqrt{N} (\hat{\beta} - \beta)
&=
\left(
    \frac{1}{N} X^T X
\right)^{-1}
\frac{1}{\sqrt{N}} X^T u
\overset{d}{\longrightarrow} \left( E(X_i X_i^T)  \right)^{-1} \times N\left( 0, E(u_i^2 X_i X_i^T) \right)\\
&= N\left( 0,
    \left( E(X_i X_i^T)  \right)^{-1}  E(u_i^2 X_i X_i^T)  \left( E(X_i X_i^T)  \right)^{-1}
\right)\\
&= N(0, V)\\
\end{align}
$$

となる。

:::{dropdown} スルツキーの定理

確率変数の行列$Y_N, Y, X_N, X \in \mathbb{R}^{N\times N}$、正則行列$C \in \mathbb{R}^{N\times N}$があるとする。

$N\to \infty$のとき

$$
X_N \xrightarrow{d} X\\
Y_N \xrightarrow{d} C\\
$$

とする。

このとき、以下の結果が成り立ち、これを **スルツキーの定理** という

1. $X_N + Y_N \xrightarrow{d} X + C$
2. $Y_N X_N \xrightarrow{d} C X$
3. $Y_N^{-1}X_N \xrightarrow{d} C^{-1} X$

:::

$V$は以下のように一致推定できる

$$
\begin{aligned}
\hat{V} & =\left[\frac{1}{N} \sum_{i=1}^N X_i X_i^T\right]^{-1} \frac{1}{N} \sum_{i=1}^N \hat{u}_i^2 X_i X_i^T\left[\frac{1}{N} \sum_{i=1}^N X_i X_i^T\right]^{-1} \\
& =\left(\frac{1}{N} X^T X\right)^{-1} \frac{1}{N} X^T \hat{U} X\left(\frac{1}{N} X^T X\right)^{-1}
\end{aligned}
$$

ただし、$\hat{U}$は対角要素に$\hat{u}_1^2, \dots, \hat{u}_N^2$を並べた対角行列である。

## バイアスとバリアンス

最小二乗推定量はすべての線形不偏推定量の中で最もバリアンスが小さい（最良である）ことを示すガウス・マルコフの定理というものがある。

```{margin} バイアスとバリアンス
統計学・機械学習の分野では誤差をバイアス（bias）とバリアンス（variance）に分けて考えることが多い。

バイアスは真の値$\theta$とサンプルを変えて推定を繰り返したときの個々の推定結果の平均$E(\hat{\theta})$との差で、
バリアンスは推定の分散$V(\hat{\theta})$である。

$$
Bias = \theta - E(\hat{\theta})\\
Variance = V(\hat{\theta})
$$
```


```{note} ガウス・マルコフの定理
各$i$について、

- $E[\varepsilon_i] = 0$
- $V[\varepsilon_i]=\sigma^2 < \infty$ が共通
- $i \neq j$のとき$E[\varepsilon_i \varepsilon_j] = 0$

を満たすとき、最小二乗推定量$\hat{\beta}_{OLS}$はBLUEになる
```


#### 不偏性

任意のパラメータの線形結合$\theta=\boldsymbol{\alpha}^\top {\boldsymbol{\beta}}$を考える。例えば$f(x_0)=x_0^\top \beta$がこの形である。

この最小二乗推定値は

$$
\hat{\theta} = \boldsymbol{\alpha}^\top \hat{\boldsymbol{\beta}}
= \boldsymbol{\alpha}^\top (\boldsymbol{X}^\top \boldsymbol{X})^{-1} \boldsymbol{X}^\top \boldsymbol{y}
$$

で、期待値をとると

$$
\begin{align}
E[\hat{\theta}]
&= E[\boldsymbol{\alpha}^\top \hat{\boldsymbol{\beta}}]\\
&= E[\boldsymbol{\alpha}^\top (\boldsymbol{X}^\top \boldsymbol{X})^{-1} \boldsymbol{X}^\top \boldsymbol{y}]\\
% &= \boldsymbol{\alpha}^\top (\boldsymbol{X}^\top \boldsymbol{X})^{-1} \boldsymbol{X}^\top E[\boldsymbol{y}]\\
&= E[\boldsymbol{\alpha^\top (X^\top X)^{-1} X^\top (X\beta + \varepsilon) }] \\
&= \boldsymbol{\alpha^\top (X^\top X)^{-1} X^\top X \beta + \alpha^\top (X^\top X)^{-1} X^\top} E[ \varepsilon ] \\
&= \boldsymbol{\alpha^\top (X^\top X)^{-1} X^\top X \beta}\\
&= \boldsymbol{\alpha}^\top \boldsymbol{\beta}
\end{align}
$$

```{margin}
※誤差項は仮定により$E[\boldsymbol{\varepsilon}]=0$であるため
```

となり（※）、$\hat{\theta}$が不偏推定量である（$E[\hat{\theta}] = \theta$）ことがわかる。


### バリアンス

$\boldsymbol{\alpha}^\top \boldsymbol{\beta}$に対して不偏のまた別の線形推定量$\boldsymbol{c}^\top \boldsymbol{y}$があるとする。

両者の差を

$$
\begin{align}
\boldsymbol{\alpha}^\top \boldsymbol{\hat{\beta}} - \boldsymbol{c}^\top \boldsymbol{y}
&= [ \boldsymbol{\alpha}^\top (\boldsymbol{X}^\top \boldsymbol{X})^{-1} \boldsymbol{X} - \boldsymbol{c}^\top ] \boldsymbol{y}\\
&=: \boldsymbol{d}^\top \boldsymbol{y}
\end{align}
$$

とおく。このとき、不偏性$E[\boldsymbol{c}^\top \boldsymbol{y}] = \boldsymbol{\alpha}^\top \boldsymbol{\beta}$から両者の差の期待値はゼロになるべきであり、

$$
E[\boldsymbol{d}^\top \boldsymbol{y}]
= \boldsymbol{d}^\top \boldsymbol{X} \boldsymbol{\beta} = 0
$$

が任意の$\boldsymbol{\beta}$について成り立たなければならないため、

$$
\boldsymbol{d}^\top \boldsymbol{X} = \boldsymbol{0}
$$

が成り立つ。


次に、2つの確率変数$X, Y$に対し

$$
V[X+Y] = V[X] + 2 \text{Cov}[X, Y] + V[Y]
$$

が成り立つから、$\boldsymbol{c}^\top \boldsymbol{y}$の分散は

$$
\begin{align}
V[\boldsymbol{c^\top y}] &= V[\boldsymbol{ \alpha^\top \hat{\beta} - d^\top y }]\\
&= V[\boldsymbol{ \alpha^\top \hat{\beta} }]
- 2 \text{Cov} [\boldsymbol{ \alpha^\top (X^\top X)^{-1} X^\top y }, \boldsymbol{ d^\top y} ]
+ V[\boldsymbol{ d^\top y}]
\end{align}
$$

と表すことができる。


$$
\text{Cov}(A, B) = E[(A - E[A])(B - E[B])^\top]
$$

$$
\text{Cov}(a^\top y, b^\top y)
= E[(a^\top y - E[a^\top y])(b^\top y - E[b^\top y])^\top]
$$

$$
\text{Cov}(a^\top y, b^\top y)
= E[(a^\top y - E[a^\top y])(b^\top y - E[b^\top y])^\top]\\
= E[(a^\top y - \alpha^\top \beta)(b^\top y - E[b^\top y])^\top]
$$

次に、2つの確率変数

$$
\boldsymbol{a^\top y} = \sum a_i y_i,
\hspace{2em}
\boldsymbol{b^\top y} = \sum b_i y_i
$$

の共分散は、誤差項$\boldsymbol{\varepsilon}$が無相関・等分散の仮定$V[\boldsymbol{\varepsilon}] = \sigma^2 \boldsymbol{I}$を満たすとき、

$$
\text{Cov}[\boldsymbol{a^\top y, b^\top y}]
= \text{Cov}[\boldsymbol{a^\top \varepsilon, b^\top \varepsilon}]
= \sum a_i b_i \sigma^2
= (\boldsymbol{a^\top b}) \sigma^2
$$

```{margin}
TODO: 解説書く
```

となることから

$$
\text{Cov}[\boldsymbol{ \alpha^\top (X^\top X)^{-1} X^\top y, d^\top y }]
= \boldsymbol{\alpha^\top (X^\top X)^{-1} X^\top d } \cdot \sigma^2
$$

となり、$\boldsymbol{d}^\top \boldsymbol{X} = \boldsymbol{0}$よりこれは0となる。

よって

$$
V[\boldsymbol{ c^\top y }] = V[\boldsymbol{ \alpha^\top \hat{\beta} }] + V[\boldsymbol{ d^\top y }]
$$

が成り立ち、分散は非負なので

$$
V[\boldsymbol{ c^\top y }] \geq V[\boldsymbol{ \alpha^\top \hat{\beta} }]
$$

を意味する。

よって$\boldsymbol{ \alpha^\top \hat{\beta} }$は最良線形不偏推定量BLUEである。

## OLS推定の幾何学的意味

OLS推定量

$$
\hat{\boldsymbol{\beta}} = (X^\top X)^{-1} X^\top \boldsymbol{y}
$$

を$\hat{\boldsymbol{y}} = X \hat{\boldsymbol{\beta}}$に代入すると

$$
\hat{\boldsymbol{y}} = \underbrace{ X (X^\top X)^{-1} X^\top }_{P} \boldsymbol{y}
= P \boldsymbol{y}
$$

つまり、ベクトル$\boldsymbol{y}$を行列$P = X (X^\top X)^{-1} X^\top$で射影したものとみなすことができる。

この行列$P$は対称行列で、$P^2=P$となる。この2つの性質を満たす行列を射影行列という。

:::{card}
$$
\begin{aligned}
P^2 & = PP \\
& =(X(X^{\top} X)^{-1} X^{\top})(X(X^{\top} X)^{-1} X^{\top}) \\
& =X(X^{\top} X)^{-1}(X^{\top} X)(X^{\top} X)^{-1} X^{\top} \\
& =X(X^{\top} X)^{-1} X^{\top}
=P
\end{aligned}
$$
:::

射影行列は、$X$の列空間$\Im X$にベクトルを正射影するという性質がある。$\boldsymbol{y}$の$\Im X$への射影が$\hat{\boldsymbol{y}}$で、垂線の足が誤差$\boldsymbol{u}$となる。

よって、最小二乗法は$\boldsymbol{y}$から$\Im X$への射影を求める操作であると捉えることができる。

## OLSとBLUE / BUE

:::{card} OLSはBLUEかBUEか？

行列$A$によって$b = Ay$のような線形結合で表現される推定量$b$は線形推定量という。OLSも$A=(X^\top X)^{-1} X^\top$とおけば同じ形になっていることがわかる。

OLSはガウス・マルコフの定理でBLUE（線形不偏推定量のなかで最良）だと示された。

[Hansen (2022)](https://doi.org/10.3982/ECTA19255) は線形制約は不要で、線形と非線形の両方のすべての不偏推定量の中で最良（BUE）だと主張した。
一方で [Pötscher & Preinerstorfer (2022)](https://doi.org/10.48550/arXiv.2203.01425) はHansen (2022)に対する批判を展開した。
[Portnoy (2022)](https://doi.org/10.1080/00031305.2022.2076743) は「一般線形モデルの場合、十分に広い分布族内のすべての分布に対して、不偏な推定量は線形でなければならない」と述べている。

Hansen(2022) の「OLSが最良の不偏推定量（BUE）」が正しいとしても、Portnoy (2022)によれば「一般線形モデルの不偏推定量は線形推定量」なので結局BLUE=BUEになり、いずれの主張も間違っていないことになる。


参考：

- 元ネタ：[Is OLS BLUE or BUE? | Statistical Horizons](https://statisticalhorizons.com/is-ols-blue-or-bue/)
- [Hansen, B. E. (2022). A Modern Gauss–Markov Theorem. Econometrica, 90(3), 1283-1294.](https://doi.org/10.3982/ECTA19255)
- [Portnoy, S. (2022). Linearity of unbiased linear model estimators. The American Statistician, (just-accepted), 1-10.](https://doi.org/10.1080/00031305.2022.2076743)
- [Pötscher, B. M., & Preinerstorfer, D. (2022). A Modern Gauss-Markov Theorem? Really?. arXiv preprint arXiv:2203.01425.](https://doi.org/10.48550/arXiv.2203.01425)
:::

## 参考文献

- 東京大学出版会『統計学入門』
- 東京大学出版会『自然科学の統計学』
- Hastie, T., Tibshirani, R., Friedman, J. H., & Friedman, J. H. (2009). The elements of statistical learning: data mining, inference, and prediction (Vol. 2, pp. 1-758). New York: springer.
- [西山慶彦, 新谷元嗣, 川口大司, & 奥井亮. (2019). 計量経済学. Yūhikaku.](https://www.yuhikaku.co.jp/books/detail/9784641053854)